In [2]:
import pandas as pd
import numpy as np

### Importación de datos

In [3]:
sv19_df = pd.read_csv('total_sv19.csv')
sv19_df = sv19_df[['CVE_MUNICIPIO', 'total']].astype(int)
sv19_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 393 entries, 0 to 392
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   CVE_MUNICIPIO  393 non-null    int64
 1   total          393 non-null    int64
dtypes: int64(2)
memory usage: 6.3 KB


In [4]:
sv20_df = pd.read_csv('total_sv20.csv')
sv20_df = sv20_df[['CVE_MUNICIPIO', 'total']].astype(int)
sv20_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 981 entries, 0 to 980
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   CVE_MUNICIPIO  981 non-null    int64
 1   total          981 non-null    int64
dtypes: int64(2)
memory usage: 15.5 KB


In [5]:
sv1920_df = pd.read_csv('sv_1920.csv')
sv1920_df = sv1920_df[['ET_ID', 'total1920']]
sv1920_df.columns = ['ET_ID', 'Total 2019-2020']
sv1920_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2469 entries, 0 to 2468
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   ET_ID            2469 non-null   int64
 1   Total 2019-2020  2469 non-null   int64
dtypes: int64(2)
memory usage: 38.7 KB


### Limpieza de los DataFrames

In [6]:
et_id = sv1920_df['ET_ID'].to_list()
sv_19_cve_municipio = sv19_df['CVE_MUNICIPIO'].to_list()
sv19_missing = [i for i in et_id if i not in sv_19_cve_municipio]
sv19_missing_df = pd.DataFrame({'CVE_MUNICIPIO': sv19_missing, 'total': np.zeros(len(sv19_missing))})
sv19_df = pd.concat([sv19_df, sv19_missing_df], ignore_index=True)
sv19_df.columns = ['CVE_MUNICIPIO', 'Total 2019']
sv19_df

,CVE_MUNICIPIO,Total 2019
0,2001,40000.0
1,2002,5000.0
2,2004,50000.0
3,4002,105000.0
4,4003,935000.0
...,...,...
2467,32054,0.0
2468,32055,0.0
2469,32056,0.0
2470,32057,0.0


In [7]:
et_id = sv1920_df['ET_ID'].to_list()
sv_20_cve_municipio = sv20_df['CVE_MUNICIPIO'].to_list()
sv20_missing = [i for i in et_id if i not in sv_20_cve_municipio]
sv20_missing_df = pd.DataFrame({'CVE_MUNICIPIO': sv20_missing, 'total': np.zeros(len(sv20_missing))})
sv20_df = pd.concat([sv20_df, sv20_missing_df], ignore_index=True)
sv20_df.columns = ['CVE_MUNICIPIO', 'Total 2020']
sv20_df

,CVE_MUNICIPIO,Total 2020
0,2001,10000.0
1,2004,10000.0
2,4001,39565000.0
3,4002,220000.0
4,4003,150000.0
...,...,...
2465,32054,0.0
2466,32055,0.0
2467,32056,0.0
2468,32057,0.0


### División en cuantiles

In [8]:
def quantiles(df: pd.DataFrame, values: str, cvs_muncs: str, variable: str='', n: int=10, groupZeros: bool=False) -> pd.DataFrame :
    
    variable = values if variable == '' else variable
    df = df.sort_values(by=values, ascending=True)
    zeros = 0
    names = [variable for _ in range(n + groupZeros)]
    bins = []
    lower = []
    upper = []
    muncs = []

    if groupZeros:
        zeros = df[values].value_counts().get(0,0)
        zeroQuantile = df.iloc[0:zeros, [0,1]]
        bins.append(0)
        lower.append(np.float64(zeroQuantile[values].iloc[0]))
        upper.append(np.float64(zeroQuantile[values].iloc[-1]))
        muncs.append(list(zeroQuantile[cvs_muncs]))

    quantiles = np.array_split(df.iloc[zeros:], n)

    for i, q in enumerate(quantiles):
        bins.append(i+1)
        lower.append(np.float64(q[values].iloc[0]))
        upper.append(np.float64(q[values].iloc[-1]))
        muncs.append(list(q[cvs_muncs]))


    return pd.DataFrame({'name':names, 'bin':bins, 'lower': lower, 'upper': upper, 'cells':muncs})


In [9]:
total_sv19_df = quantiles(sv19_df, 'Total 2019', 'CVE_MUNICIPIO', n=10)
total_sv19_df

/usr/local/lib/python3.9/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


,name,bin,lower,upper,cells
0,Total 2019,1,0.0,0.0,"[13023, 19017, 19018, 19020, 19021, 19022, 190..."
1,Total 2019,2,0.0,0.0,"[20071, 20070, 20069, 20041, 20045, 20048, 200..."
2,Total 2019,3,0.0,0.0,"[15090, 15091, 15093, 15113, 15114, 15115, 160..."
3,Total 2019,4,0.0,0.0,"[26005, 26006, 26007, 26008, 26009, 26010, 250..."
4,Total 2019,5,0.0,0.0,"[20439, 20442, 20374, 20443, 20446, 20447, 204..."
5,Total 2019,6,0.0,0.0,"[20221, 12035, 21183, 31054, 20282, 31094, 200..."
6,Total 2019,7,0.0,0.0,"[20204, 28019, 21030, 20518, 20465, 20341, 290..."
7,Total 2019,8,0.0,0.0,"[5018, 5017, 5027, 7014, 7017, 7020, 7093, 708..."
8,Total 2019,9,0.0,5160000.0,"[1001, 8029, 8027, 4004, 18009, 25017, 30123, ..."
9,Total 2019,10,5275000.0,908500000.0,"[30116, 30034, 30145, 30071, 7088, 21029, 3016..."


In [10]:
total_sv20_df = quantiles(sv20_df, 'Total 2020', 'CVE_MUNICIPIO', n=10)
total_sv20_df

/usr/local/lib/python3.9/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


,name,bin,lower,upper,cells
0,Total 2020,1,0.0,0.0,"[13024, 19018, 19020, 19021, 19022, 19023, 190..."
1,Total 2020,2,0.0,0.0,"[20072, 20090, 20071, 20069, 20045, 20048, 200..."
2,Total 2020,3,0.0,0.0,"[15090, 15091, 15092, 15094, 15114, 15115, 151..."
3,Total 2020,4,0.0,0.0,"[26004, 26005, 26006, 26007, 26008, 26009, 260..."
4,Total 2020,5,0.0,0.0,"[20438, 20439, 20442, 20374, 20443, 20446, 204..."
5,Total 2020,6,0.0,0.0,"[13023, 7085, 7083, 7078, 7075, 7064, 7058, 70..."
6,Total 2020,7,0.0,1680000.0,"[11003, 11007, 11008, 11009, 11010, 11011, 110..."
7,Total 2020,8,1690000.0,7315000.0,"[8051, 21049, 17002, 21101, 20223, 20288, 2102..."
8,Total 2020,9,7325000.0,21425000.0,"[30157, 31058, 29040, 20394, 29008, 20486, 301..."
9,Total 2020,10,21430000.0,955155000.0,"[30100, 24014, 30195, 10030, 7079, 7069, 30204..."


In [11]:
total_sv1920_df = quantiles(sv1920_df, 'Total 2019-2020', 'ET_ID', n=10)
total_sv1920_df

/usr/local/lib/python3.9/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


,name,bin,lower,upper,cells
0,Total 2019-2020,1,0.0,0.000000e+00,"[13027, 19018, 19020, 19021, 19022, 19023, 190..."
1,Total 2019-2020,2,0.0,0.000000e+00,"[20072, 20090, 20071, 20069, 20045, 20048, 200..."
2,Total 2019-2020,3,0.0,0.000000e+00,"[15090, 15091, 15092, 15094, 15114, 15115, 151..."
3,Total 2019-2020,4,0.0,0.000000e+00,"[26004, 26005, 26006, 26007, 26008, 26009, 260..."
4,Total 2019-2020,5,0.0,0.000000e+00,"[20438, 20439, 20442, 20374, 20443, 20446, 204..."
5,Total 2019-2020,6,0.0,5.000000e+03,"[13023, 7108, 7106, 7098, 7094, 7093, 7086, 70..."
6,Total 2019-2020,7,5000.0,1.835000e+06,"[18017, 19038, 16058, 28028, 21140, 21166, 260..."
7,Total 2019-2020,8,1840000.0,8.415000e+06,"[31003, 20093, 20456, 20304, 31072, 20237, 200..."
8,Total 2019-2020,9,8425000.0,2.654000e+07,"[10015, 20520, 30150, 20274, 8041, 21025, 2041..."
9,Total 2019-2020,10,27265000.0,1.863655e+09,"[29034, 16065, 12057, 7091, 30127, 10019, 1003..."


In [12]:
## |Sembrando vida total 2019, q00, 0.0:0.0|cells

def varByQuantil(df: pd.DataFrame, values: str, cvs_muncs: str, variable: str='', n: int=10, groupZeros: bool=False) -> pd.DataFrame :
    
    variable = values if variable == '' else variable
    df = df.sort_values(by=values, ascending=True)
    zeros = 0
    names = []
    muncs = []

    if groupZeros:
        zeros = df[values].value_counts().get(0,0)
        zeroQuantile = df.iloc[0:zeros, [0,1]]
        name = variable + ', q00, 0.0:0.0'
        names.append(name)
        muncs.append(list(zeroQuantile[cvs_muncs]))

    quantiles = np.array_split(df.iloc[zeros:], n)

    for i, q in enumerate(quantiles):
        name = variable + ', q' + '{:02}'.format(i+1) + ', ' + str(np.float64(q[values].iloc[0])) + ':' + str(np.float64(q[values].iloc[-1]))
        names.append(name)
        muncs.append(list(q[cvs_muncs]))


    return pd.DataFrame({'name':names, 'cells':muncs})


In [13]:
total_sv19_df = varByQuantil(sv19_df, 'Total 2019', 'CVE_MUNICIPIO', n=10, variable='Sembrando vida total 2019')
total_sv19_df

/usr/local/lib/python3.9/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


,name,cells
0,"Sembrando vida total 2019, q01, 0.0:0.0","[13023, 19017, 19018, 19020, 19021, 19022, 190..."
1,"Sembrando vida total 2019, q02, 0.0:0.0","[20071, 20070, 20069, 20041, 20045, 20048, 200..."
2,"Sembrando vida total 2019, q03, 0.0:0.0","[15090, 15091, 15093, 15113, 15114, 15115, 160..."
3,"Sembrando vida total 2019, q04, 0.0:0.0","[26005, 26006, 26007, 26008, 26009, 26010, 250..."
4,"Sembrando vida total 2019, q05, 0.0:0.0","[20439, 20442, 20374, 20443, 20446, 20447, 204..."
5,"Sembrando vida total 2019, q06, 0.0:0.0","[20221, 12035, 21183, 31054, 20282, 31094, 200..."
6,"Sembrando vida total 2019, q07, 0.0:0.0","[20204, 28019, 21030, 20518, 20465, 20341, 290..."
7,"Sembrando vida total 2019, q08, 0.0:0.0","[5018, 5017, 5027, 7014, 7017, 7020, 7093, 708..."
8,"Sembrando vida total 2019, q09, 0.0:5160000.0","[1001, 8029, 8027, 4004, 18009, 25017, 30123, ..."
9,"Sembrando vida total 2019, q10, 5275000.0:9085...","[30116, 30034, 30145, 30071, 7088, 21029, 3016..."


In [14]:
total_sv20_df = varByQuantil(sv20_df, 'Total 2020', 'CVE_MUNICIPIO', n=10, variable='Sembrando vida total 2020')
total_sv20_df

/usr/local/lib/python3.9/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


,name,cells
0,"Sembrando vida total 2020, q01, 0.0:0.0","[13024, 19018, 19020, 19021, 19022, 19023, 190..."
1,"Sembrando vida total 2020, q02, 0.0:0.0","[20072, 20090, 20071, 20069, 20045, 20048, 200..."
2,"Sembrando vida total 2020, q03, 0.0:0.0","[15090, 15091, 15092, 15094, 15114, 15115, 151..."
3,"Sembrando vida total 2020, q04, 0.0:0.0","[26004, 26005, 26006, 26007, 26008, 26009, 260..."
4,"Sembrando vida total 2020, q05, 0.0:0.0","[20438, 20439, 20442, 20374, 20443, 20446, 204..."
5,"Sembrando vida total 2020, q06, 0.0:0.0","[13023, 7085, 7083, 7078, 7075, 7064, 7058, 70..."
6,"Sembrando vida total 2020, q07, 0.0:1680000.0","[11003, 11007, 11008, 11009, 11010, 11011, 110..."
7,"Sembrando vida total 2020, q08, 1690000.0:7315...","[8051, 21049, 17002, 21101, 20223, 20288, 2102..."
8,"Sembrando vida total 2020, q09, 7325000.0:2142...","[30157, 31058, 29040, 20394, 29008, 20486, 301..."
9,"Sembrando vida total 2020, q10, 21430000.0:955...","[30100, 24014, 30195, 10030, 7079, 7069, 30204..."


In [15]:
total_sv1920_df = varByQuantil(sv1920_df, 'Total 2019-2020', 'ET_ID', n=10, variable='Sembrando vida total 2019-2020')
total_sv1920_df

/usr/local/lib/python3.9/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


,name,cells
0,"Sembrando vida total 2019-2020, q01, 0.0:0.0","[13027, 19018, 19020, 19021, 19022, 19023, 190..."
1,"Sembrando vida total 2019-2020, q02, 0.0:0.0","[20072, 20090, 20071, 20069, 20045, 20048, 200..."
2,"Sembrando vida total 2019-2020, q03, 0.0:0.0","[15090, 15091, 15092, 15094, 15114, 15115, 151..."
3,"Sembrando vida total 2019-2020, q04, 0.0:0.0","[26004, 26005, 26006, 26007, 26008, 26009, 260..."
4,"Sembrando vida total 2019-2020, q05, 0.0:0.0","[20438, 20439, 20442, 20374, 20443, 20446, 204..."
5,"Sembrando vida total 2019-2020, q06, 0.0:5000.0","[13023, 7108, 7106, 7098, 7094, 7093, 7086, 70..."
6,"Sembrando vida total 2019-2020, q07, 5000.0:18...","[18017, 19038, 16058, 28028, 21140, 21166, 260..."
7,"Sembrando vida total 2019-2020, q08, 1840000.0...","[31003, 20093, 20456, 20304, 31072, 20237, 200..."
8,"Sembrando vida total 2019-2020, q09, 8425000.0...","[10015, 20520, 30150, 20274, 8041, 21025, 2041..."
9,"Sembrando vida total 2019-2020, q10, 27265000....","[29034, 16065, 12057, 7091, 30127, 10019, 1003..."


In [16]:
## total_sv19_df.to_json('sv19.json', index=False, orient='records')
## total_sv20_df.to_json('sv20.json', index=False, orient='records')
## total_sv1920_df.to_json('sv1920.json', index=False, orient='records')

In [17]:
covariables19_df = pd.read_csv("covariables_sv.csv", dtype={'especievalida':str, 'min':str, 'max':str, 'ET_ID':int})[:2469]
covariables19_df['especievalida'] = covariables19_df['especievalida'].replace(regex=['\APolitic01.p', ' .*'], value='')
covariables19_df = covariables19_df[['especievalida', 'ET_ID']]
covariables19_df

,especievalida,ET_ID
0,00,15030
1,00,15069
2,00,20103
3,00,20157
4,00,20215
...,...,...
2464,09,23004
2465,00,4006
2466,10,7059
2467,10,23002


In [18]:
for i in range(0,11):
    especievalida = '{:02}'.format(i)
    q = covariables19_df['especievalida'].value_counts().get(especievalida,0)
    print(especievalida, ": ", q)

00 :  2079
01 :  39
02 :  38
03 :  39
04 :  40
05 :  39
06 :  39
07 :  39
08 :  39
09 :  39
10 :  39


In [19]:
q_cov = [[],[],[],[],[],[],[],[],[],[],[]]
for i in range(0,11):
    especievalida = '{:02}'.format(i)
    
    for j in range(0,len(covariables19_df)):
        if covariables19_df['especievalida'][j] == especievalida:
            q_cov[i].append(covariables19_df['ET_ID'][j])
    print("q_cov", especievalida, ": ", q_cov[i])

q_cov 00 :  [15030, 15069, 20103, 20157, 20215, 20389, 20393, 21013, 21038, 21097, 21117, 21118, 21144, 26007, 20411, 20087, 15044, 20192, 11044, 32057, 26016, 20063, 32006, 21151, 20538, 21125, 26070, 29054, 20174, 26060, 20132, 29056, 29060, 20315, 20555, 20013, 15100, 32013, 32005, 20539, 14098, 20069, 21090, 8053, 20542, 20077, 29058, 24049, 20342, 20109, 29048, 20049, 20338, 15058, 29008, 20268, 20403, 20033, 26048, 15108, 29053, 26071, 29057, 20145, 29044, 29042, 10036, 15031, 15022, 15125, 20375, 20387, 20007, 20390, 26026, 20061, 26047, 21181, 15076, 20227, 24025, 16011, 1004, 32037, 15053, 20488, 13069, 32053, 15011, 1011, 15055, 21146, 20452, 32016, 29011, 20301, 32035, 26064, 9014, 21041, 20106, 20238, 29059, 9015, 29045, 20360, 15027, 26025, 20144, 26046, 32054, 21182, 29029, 20241, 15059, 20380, 20493, 32032, 14070, 15073, 20399, 20101, 20556, 29019, 29051, 15012, 21128, 29049, 26056, 21133, 26017, 20096, 1009, 29016, 21171, 32025, 17031, 26004, 32039, 21126, 20165, 8024, 

In [20]:
quantiles_covariables19_df = quantiles(covariables19_df, 'especievalida', 'ET_ID', 10, True)
quantiles_covariables19_df

/usr/local/lib/python3.9/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


,name,bin,lower,upper,cells
0,10,1,0.0,10.0,"[15030, 20229, 12064, 20549, 16081, 31065, 210..."


In [21]:
for i in range(0, 10):
    q_total_sv19 = total_sv19_df['cells'][i]
    res = [x for x in q_cov[i + 1] + q_total_sv19 if x not in q_cov[i + 1] or x not in q_total_sv19]
    print("Dif en q", i, ":", res)

Dif en q 0 : [21034, 9006, 19046, 9002, 8005, 15122, 9011, 8004, 9007, 8045, 19025, 8032, 20490, 2002, 15020, 30026, 30191, 7113, 2004, 9010, 30036, 30139, 30016, 5035, 12013, 21213, 17020, 5002, 7034, 5007, 14022, 6009, 30009, 2001, 7008, 7019, 14061, 7097, 7107, 13023, 19017, 19018, 19020, 19021, 19022, 19023, 19024, 19026, 19027, 19028, 19029, 19030, 19031, 19032, 19033, 19016, 19034, 19015, 19013, 18016, 18018, 18020, 19001, 19002, 19003, 19004, 19005, 19006, 19007, 19008, 19009, 19010, 19011, 19012, 19014, 19035, 19036, 19037, 20015, 20017, 20019, 20020, 20021, 20022, 20023, 20024, 20026, 20027, 20028, 20029, 20032, 20033, 20034, 20013, 20012, 20011, 20009, 19039, 19040, 19041, 19042, 19043, 19044, 19045, 18015, 19047, 19049, 19050, 19051, 20001, 20002, 20004, 20007, 19048, 20035, 18014, 18012, 16069, 16070, 16071, 16072, 16073, 16074, 16075, 16076, 16078, 16079, 16080, 16081, 16083, 16084, 16085, 16068, 16086, 16067, 16062, 16040, 16041, 16042, 16043, 16044, 16045, 16046, 16047, 

In [22]:
covariables20_df = pd.read_csv("covariables_sv.csv", dtype={'especievalida':str, 'min':str, 'max':str, 'ET_ID':int})[2469:4938]
covariables20_df['especievalida'] = covariables20_df['especievalida'].replace(regex=['\APolitic02.p', ' .*'], value='')
covariables20_df = covariables20_df[['especievalida', 'ET_ID']]
covariables20_df

,especievalida,ET_ID
2469,00,15030
2470,00,15069
2471,00,20103
2472,00,20157
2473,02,20215
...,...,...
4933,10,23004
4934,10,4006
4935,10,7059
4936,10,23002


In [23]:
for i in range(0,11):
    especievalida = '{:02}'.format(i)
    q = covariables20_df['especievalida'].value_counts().get(especievalida,0)
    print(especievalida, ": ", q)

00 :  1489
01 :  97
02 :  98
03 :  99
04 :  98
05 :  98
06 :  98
07 :  98
08 :  98
09 :  98
10 :  98


In [24]:
q_cov20 = [[],[],[],[],[],[],[],[],[],[],[]]
for i in range(0,11):
    especievalida = '{:02}'.format(i)
    
    for j in range(2469,4938):
        if covariables20_df['especievalida'][j] == especievalida:
            q_cov20[i].append(covariables20_df['ET_ID'][j])
    print("q_cov20", especievalida, ": ", q_cov20[i])

q_cov20 00 :  [15030, 15069, 20103, 20157, 20389, 20393, 21013, 21038, 21097, 21117, 21118, 21144, 26007, 20411, 20087, 15044, 20192, 11044, 32057, 26016, 20063, 32006, 21151, 20538, 21125, 26070, 29054, 20174, 26060, 20132, 29056, 29060, 20315, 20555, 20013, 21034, 15100, 32013, 32005, 20539, 14098, 20069, 21090, 8053, 20542, 20077, 29058, 24049, 20342, 20109, 29048, 20049, 20338, 20268, 20403, 20033, 26048, 15108, 9006, 29053, 26071, 29057, 20145, 29044, 29042, 10036, 15031, 15022, 15125, 20375, 20387, 20007, 20390, 26026, 20061, 26047, 15076, 20227, 24025, 16011, 1004, 32037, 15053, 20488, 13069, 32053, 15011, 1011, 15055, 20452, 32016, 29011, 20301, 32035, 26064, 9014, 21041, 20238, 29059, 9015, 29045, 20360, 15027, 26025, 26046, 32054, 21182, 29029, 20241, 15059, 20380, 32032, 14070, 19046, 15073, 20399, 20101, 29019, 8005, 29051, 15012, 21128, 29049, 26056, 21133, 26017, 20096, 1009, 15122, 21171, 32025, 17031, 26004, 32039, 20165, 8024, 21136, 5010, 26020, 15006, 15061, 26065, 1

In [25]:
for i in range(0, 10):
    q_total_sv20 = total_sv20_df['cells'][i]
    res = [x for x in q_cov20[i + 1] + q_total_sv20 if x not in q_cov20[i + 1] or x not in q_total_sv20]
    print("Dif en q", i, ":", res)

Dif en q 0 : [15058, 21119, 8037, 9017, 9002, 26055, 21140, 15024, 9011, 15070, 10007, 20385, 29005, 8004, 28009, 9007, 8045, 19025, 9003, 8032, 29043, 9016, 24056, 9005, 15109, 21166, 20444, 30138, 15033, 15020, 30206, 12050, 13083, 30026, 30146, 28038, 30191, 7113, 30140, 19019, 2004, 9010, 30036, 30139, 9008, 8060, 13030, 21160, 15057, 24028, 30107, 21068, 30162, 30087, 28032, 13004, 5035, 30211, 16087, 30105, 20357, 10022, 21156, 8043, 30177, 32026, 9009, 17020, 30015, 11030, 31036, 28021, 16058, 5002, 16014, 16059, 28028, 16066, 31006, 31080, 8017, 7034, 5007, 28004, 20184, 12078, 2001, 12075, 7008, 16006, 7019, 28012, 14061, 7097, 18017, 19038, 8031, 13024, 19018, 19020, 19021, 19022, 19023, 19024, 19026, 19027, 19028, 19029, 19030, 19031, 19032, 19033, 19034, 19017, 19035, 19016, 19014, 18018, 18020, 19001, 19002, 19003, 19004, 19005, 19006, 19007, 19008, 19009, 19010, 19011, 19012, 19013, 19015, 19036, 19037, 19039, 20017, 20019, 20020, 20021, 20022, 20023, 20024, 20026, 20027,